# 🧠 Phi-3 RAG — Bengali Q&A + Quiz Generation
### NCTB Class 3 AI Tutor | Group 11 — BUET CSE

**এই notebook এ যা হবে:**
1. আগের notebook এর FAISS index load করা
2. Phi-3 Mini model download ও load করা (Kaggle T4 GPU তে চলবে)
3. Bengali/English প্রশ্ন → NCTB context retrieve → Phi-3 উত্তর
4. AI দিয়ে Quiz generate করা
5. Interactive chat loop test করা

---
**⚠️ Setup:**
- আগের OCR notebook এর output dataset হিসেবে add করো
- GPU T4 x2 ON রাখো
- Internet ON রাখো (model download এর জন্য)

## Step 1: Install Libraries

In [13]:
import subprocess
# আগের OCR notebook এর output copy করো
subprocess.run(['cp', '-r', 
    '/kaggle/input/notebooks/sharbanichowdhury/nctb-class3-bengali-ocr', 
    '/kaggle/working/'])

CompletedProcess(args=['cp', '-r', '/kaggle/input/notebooks/sharbanichowdhury/nctb-class3-bengali-ocr', '/kaggle/working/'], returncode=0)

In [14]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q sentence-transformers faiss-cpu
!pip install -q langchain

print('✅ সব install হয়েছে!')

✅ সব install হয়েছে!


## Step 2: FAISS Index Load করা (আগের notebook থেকে)

In [15]:
import os, pickle, json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch

# ── Paths ──────────────────────────────────────────────────────
# আগের notebook এর output Kaggle Dataset হিসেবে add করতে হবে
# Dataset name যদি 'nctb-rag-output' হয়:
BASE =  '/kaggle/input/notebooks/sharbanichowdhury/nctb-class3-bengali-ocr'  # ← তোমার dataset name অনুযায়ী change করো

# Fallback: working directory তে থাকলে সেখান থেকে নাও
if not os.path.exists(BASE):
    # BASE = '/kaggle/working
    BASE =  '/kaggle/working/'
    print('⚠️  Input dataset পাওয়া যায়নি — working directory থেকে load হচ্ছে')

FAISS_PATH  = f'{BASE}/faiss_index/bangla_class3.faiss'
CHUNKS_PATH = f'{BASE}/faiss_index/chunks.pkl'

# Load FAISS index
print('🔄 FAISS index load হচ্ছে...')
INDEX = faiss.read_index(FAISS_PATH)

# Load text chunks
with open(CHUNKS_PATH, 'rb') as f:
    CHUNKS = pickle.load(f)

print(f'✅ Index loaded!')
print(f'   Vectors: {INDEX.ntotal}')
print(f'   Chunks: {len(CHUNKS)}')
print(f'   Sample chunk: {CHUNKS[5][:150]}')

🔄 FAISS index load হচ্ছে...
✅ Index loaded!
   Vectors: 190
   Chunks: 190
   Sample chunk: । সামাজিক ও অর্থনৈতিক স্তর এবং ধর্ম-বর্ণ কিংবা লৈঙ্গিক পরিচয় কোনো শিশুর শিক্ষাগ্রহণের পথে যাতে বাধা না হয়ে দাঁড়ায় এ বিষয়েও বিশেষ দৃষ্টি রাখা হয়ে


## Step 3: Embedding Model Load (Retrieval এর জন্য)

In [16]:
print('🔄 Embedding model load হচ্ছে...')

EMBED_MODEL = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

def retrieve(question, k=3):
    """Student এর question এর জন্য relevant NCTB chunks খুঁজে আনা।"""
    q_emb = EMBED_MODEL.encode(
        [question],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')
    
    scores, indices = INDEX.search(q_emb, k)
    
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx >= 0:
            results.append({
                'text': CHUNKS[idx],
                'score': float(score)
            })
    return results

# Quick test
test = retrieve('বন্ধু কাকে বলে?')
print(f'\n✅ Retrieval test:')
for i, r in enumerate(test, 1):
    print(f'  [{i}] score={r["score"]:.3f}: {r["text"][:120]}...')

🔄 Embedding model load হচ্ছে...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ Retrieval test:
  [1] score=0.547: জিজ্ঞেস করলেন তুমি কী করবে? রাশেদ বলল ? অঙ্ক দৌড় ও মোরগ লড়াই করব ক্লাসের সবাই ভাবছিল , রাশেদ পারবে তো ! নোমান স্যার বল...
  [2] score=0.520: এবার একটা ছক আঁকি | ছকটিতে নিজের ভালো লাগার কথা লিখি স্যার বোর্ডে একটি ছক আঁকলেন বললেন , আমার মতো করে তোমরাও ছকটি আঁকো...
  [3] score=0.519: (ঘন মন)
পড়ি ছুটির দিন ! ঘুমাচ্ছিলাম | হঠাৎ শুনি মিউ মিউ শব্দ জেগে উঠে দেখি ঘরের ভেতর ছোট্ট একটা বিড়ালছানা / আমি জিজ্ঞে...


## Step 4: Phi-3 Mini Load করা

**Phi-3 Mini (3.8B)** — Microsoft এর lightweight model।  
Kaggle T4 GPU তে 4-bit quantization দিয়ে চলবে (~4GB VRAM)।

In [17]:
# Hugging Face cache সম্পূর্ণ clear করো
import shutil, os

hf_cache = os.path.expanduser('~/.cache/huggingface')
shutil.rmtree(hf_cache, ignore_errors=True)
print('✅ Cache cleared!')

# এবার Phi-3.5 use করো — Phi-3 এর updated version, same size
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

MODEL_ID = 'microsoft/Phi-3.5-mini-instruct'  # Phi-3 → Phi-3.5 (bug fixed)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID)

print('🔄 Phi-3.5 Mini load হচ্ছে (~5 min)...')
PHI3 = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.float16,
)

PHI3.eval()
print(f'✅ Ready! Memory: {torch.cuda.memory_allocated()//1024**2} MB')

✅ Cache cleared!


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

✅ Ready! Memory: 2959 MB


## Step 5: RAG Generation Function

In [18]:
def generate_answer(question, mode='chat', max_new_tokens=300):
    """
    Student এর question এর জন্য NCTB-grounded উত্তর generate করা।
    
    mode:
      'chat' → সহজ Bengali উত্তর
      'quiz' → MCQ questions JSON format এ
      'explain' → বিস্তারিত ব্যাখ্যা
    """
    # Step 1: Relevant context retrieve করো
    chunks = retrieve(question, k=3)
    context = '\n\n'.join(
        f'[অংশ {i+1}]: {c["text"]}'
        for i, c in enumerate(chunks)
    )
    
    # Step 2: Mode অনুযায়ী prompt তৈরি
    if mode == 'chat':
        system = (
            'তুমি একটি AI শিক্ষক। বাংলাদেশের তৃতীয় শ্রেণির ছাত্রছাত্রীদের '
            'NCTB পাঠ্যবই অনুযায়ী সাহায্য করো। '
            'সহজ, বন্ধুত্বপূর্ণ বাংলায় ছোট উত্তর দাও। '
            'শুধু নিচের বইয়ের অংশ থেকে উত্তর দাও।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context}\n\nপ্রশ্ন: {question}'

    elif mode == 'quiz':
        system = (
            'তুমি একজন শিক্ষক। NCTB পাঠ্যবইয়ের তথ্য থেকে '
            'তৃতীয় শ্রেণির ছাত্রদের জন্য MCQ তৈরি করো। '
            'শুধু JSON format এ output দাও, অন্য কিছু লিখবে না।'
        )
        user = (
            f'পাঠ্যবইয়ের অংশ:\n{context}\n\n'
            f'"{question}" বিষয়ে ৩টি সহজ MCQ তৈরি করো।\n'
            'Format:\n'
            '{"questions": [{"question": "...", '
            '"options": ["ক) ...", "খ) ...", "গ) ...", "ঘ) ..."], '
            '"correct": "ক", "explanation": "..."}]}'
        )

    elif mode == 'explain':
        system = (
            'তুমি একটি AI শিক্ষক। NCTB পাঠ্যবই অনুযায়ী '
            'তৃতীয় শ্রেণির ছাত্রদের জন্য বিষয়টি সহজভাবে ব্যাখ্যা করো। '
            'উদাহরণ দিয়ে বোঝাও।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context}\n\nব্যাখ্যা করো: {question}'

    # Step 3: Phi-3 format এ prompt তৈরি
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user',   'content': user}
    ]
    
    prompt = TOKENIZER.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Step 4: Tokenize
    inputs = TOKENIZER(
        prompt,
        return_tensors='pt',
        truncation=True,
        max_length=2048
    ).to(PHI3.device)
    
    # Step 5: Generate
    with torch.no_grad():
        output = PHI3.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,     # Low temp = more focused/accurate
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=TOKENIZER.eos_token_id
        )
    
    # Step 6: Decode — শুধু নতুন generated text
    generated = output[0][inputs['input_ids'].shape[1]:]
    answer = TOKENIZER.decode(generated, skip_special_tokens=True).strip()
    
    return answer, context  # context ও return করি debug এর জন্য


print('✅ RAG generation function ready!')

✅ RAG generation function ready!


## Step 6: Bengali Q&A Test

In [19]:
# Test questions — Class 3 বাংলা বই থেকে
test_questions = [
    'তোমার বন্ধু কে?',
    'পরিবারে কারা থাকে?',
    'বাংলাদেশের কথা বলো।',
    'What is this lesson about?',  # English test
]

print('=' * 60)
print('🧪 Bengali Q&A Test — Phi-3 + NCTB RAG')
print('=' * 60)

for q in test_questions:
    print(f'\n❓ প্রশ্ন: {q}')
    print('🤔 উত্তর generate হচ্ছে...')
    
    answer, ctx = generate_answer(q, mode='chat')
    
    print(f'🤖 AI উত্তর:\n{answer}')
    print(f'\n📚 Retrieved context (প্রথম ১০০ chars): {ctx[:100]}...')
    print('-' * 60)

🧪 Bengali Q&A Test — Phi-3 + NCTB RAG

❓ প্রশ্ন: তোমার বন্ধু কে?
🤔 উত্তর generate হচ্ছে...
🤖 AI উত্তর:
রাশেদ

📚 Retrieved context (প্রথম ১০০ chars): [অংশ 1]: জিজ্ঞেস করলেন তুমি কী করবে? রাশেদ বলল ? অঙ্ক দৌড় ও মোরগ লড়াই করব ক্লাসের সবাই ভাবছিল , রা...
------------------------------------------------------------

❓ প্রশ্ন: পরিবারে কারা থাকে?
🤔 উত্তর generate হচ্ছে...
🤖 AI উত্তর:
পাঠ কী করব আর পাঁছেন

পাঠ ১৭ থাকে,

পৃষ্ঠা 106 

পাঠ আর পাঁছেন

পরিবারে কারা থাকে!

📚 Retrieved context (প্রথম ১০০ chars): [অংশ 1]: জিজ্ঞেস করলেন তুমি কী করবে? রাশেদ বলল ? অঙ্ক দৌড় ও মোরগ লড়াই করব ক্লাসের সবাই ভাবছিল , রা...
------------------------------------------------------------

❓ প্রশ্ন: বাংলাদেশের কথা বলো।
🤔 উত্তর generate হচ্ছে...
🤖 AI উত্তর:
বাংলাদেশের ছোট উত্তর:

প্রফেসর রবিউল কবীর চৌধুরী চেয়ারম্যান, জাতীয় শিক্ষাক্রম ও পাঠ্যপুস্তক বোর্ড, আজিকার শিশু সুফিয়া কামাল

(স)-কে হত্যা করার ঘোষণা দেয় তখন মুহাম্মদ (স) মক্কা থেকে হিজরত করেন, আবু বকর (রা)

পাঠ ১৯, C_r -- _$[4_

📚 Retrieved context (প্রথম ১

## Step 7: Quiz Generation Test

In [20]:
import json as json_lib
import re

def generate_quiz(topic, num_q=3):
    """Topic এর উপর MCQ quiz generate করা।"""
    answer, _ = generate_answer(topic, mode='quiz', max_new_tokens=500)
    
    # JSON parse করার চেষ্টা
    try:
        # JSON block extract করো
        json_match = re.search(r'\{.*\}', answer, re.DOTALL)
        if json_match:
            quiz_data = json_lib.loads(json_match.group())
            return quiz_data
    except:
        pass
    
    # JSON parse না হলে raw text return করো
    return {'raw': answer}


def display_quiz(quiz_data):
    """Quiz সুন্দরভাবে display করা।"""
    if 'raw' in quiz_data:
        print(quiz_data['raw'])
        return
    
    questions = quiz_data.get('questions', [])
    for i, q in enumerate(questions, 1):
        print(f'\n{i}. {q.get("question", "")}')
        for opt in q.get('options', []):
            print(f'   {opt}')
        print(f'   ✅ সঠিক উত্তর: {q.get("correct", "")}')
        if q.get('explanation'):
            print(f'   💡 ব্যাখ্যা: {q["explanation"]}')


# Test
print('=' * 60)
print('📝 Quiz Generation Test')
print('=' * 60)

topics = ['বন্ধুত্ব', 'পরিবার']

for topic in topics:
    print(f'\n🎯 Topic: {topic}')
    print('Quiz generate হচ্ছে...')
    quiz = generate_quiz(topic)
    display_quiz(quiz)
    print('-' * 60)

📝 Quiz Generation Test

🎯 Topic: বন্ধুত্ব
Quiz generate হচ্ছে...
{
  "questions": [
    {
      "question": "আনন্দের দিন আমার পাঠের অংশ অভাগে কত পৃষ্ঠা পর্যালন হয়?",
      "options": ["ক) 41", "খ) 35", "গ) 30", "ঘ) 45"],
      "correct": "ক) 41",
      "explan_tion": "আনন্দের দিন উদযotted পাঠের অংশে পৃষ্ঠা 41 পর্যালন হয়."
    },
    {
      "question": "এবার একটা ছক আঁকলেন 'নিজের ভালো লাগার কথা' চার্নিশ করেন, এখনোর ছাত্র কত পৃষ্ঠা হয়?",
      "options": ["ক) 41", "খ) 35", "গ) 38", "ঘ) 40"],
      "correct": "ক) 41",
      "explan_tion": "এবার একটা ছক আঁকলেন 'নিজের ভালো লাগার কথা' চার্নিশ করেন, এখনোর ছাত্র পৃষ্�
------------------------------------------------------------

🎯 Topic: পরিবার
Quiz generate হচ্ছে...

1. পাঠ্যবইয়ের পরিবার 'পৃষ্ঠা' অংশের পরিমানিত পরিশMapping কি?
   ক) পৃষ্ঠা 8
   খ) পৃষ্ঠা 35
   গ) পৃষ্ঠা 106
   ঘ) পাঠ
   ✅ সঠিক উত্তর: গ
   💡 ব্যাখ্যা: পরিবার 'পৃষ্ঠা' অংশের পরিমানিত পরিশMapping হল 'পৃষ্ঠা 106', এরhalbonoon আমি 'পৃষ্ঠা 35' অংশের পরিশMapping বলিতেই পরিবার হত

## Step 8: Interactive Chat Loop (Full Pipeline Test)

In [21]:
def detect_language(text):
    """Bengali নাকি English সেটা detect করা।"""
    bn_chars = sum(1 for c in text if '\u0980' <= c <= '\u09FF')
    return 'bengali' if bn_chars > len(text) * 0.2 else 'english'

def ai_tutor_response(student_input, conversation_history=[]):
    """
    Full AI tutor pipeline:
    Student input → Language detect → RAG → Phi-3 → Response
    """
    lang = detect_language(student_input)
    
    # Quiz request detect করা
    quiz_keywords = ['quiz', 'প্রশ্ন করো', 'পরীক্ষা', 'mcq', 'কুইজ']
    is_quiz = any(kw in student_input.lower() for kw in quiz_keywords)
    
    if is_quiz:
        # Topic extract করো এবং quiz generate করো
        topic = student_input.replace('quiz', '').replace('কুইজ', '').replace('প্রশ্ন করো', '').strip()
        if not topic:
            topic = 'বাংলা পাঠ'
        answer, ctx = generate_answer(topic, mode='quiz', max_new_tokens=500)
    else:
        answer, ctx = generate_answer(student_input, mode='chat')
    
    return {
        'answer': answer,
        'language': lang,
        'mode': 'quiz' if is_quiz else 'chat',
        'context_used': ctx[:200] + '...'
    }


# Simulate a conversation
print('=' * 60)
print('💬 AI Tutor — Simulated Conversation')
print('=' * 60)

conversations = [
    'আমাকে বন্ধুত্ব সম্পর্কে বলো।',
    'বন্ধুত্ব নিয়ে কুইজ দাও।',
    'Tell me about family.',
]

for msg in conversations:
    print(f'\n👦 Student: {msg}')
    response = ai_tutor_response(msg)
    print(f'🤖 AI Tutor ({response["mode"]} | {response["language"]}):')
    print(response['answer'])
    print('-' * 60)

💬 AI Tutor — Simulated Conversation

👦 Student: আমাকে বন্ধুত্ব সম্পর্কে বলো।
🤖 AI Tutor (chat | bengali):
রাশেদ বলল!
------------------------------------------------------------

👦 Student: বন্ধুত্ব নিয়ে কুইজ দাও।
🤖 AI Tutor (quiz | bengali):
{
  "questions": [
    {
      "question": "এবার একটা ছক আঁকিতে নিজের ভালো লাগার কথা লি�riz করেন, তুমি কী করবে?\nA) খুব ভালো লাগল রাশেদ gৱ\nB) ঘুর পাঠ পরে\nC) ব্যাঙের সাজা\nD) পৃষ্ঠা 35\n" ,
      "options": ["A) খুব ভালো লাগল রাশেদ gৱ", "B) ঘুর পাঠ পরে", "C) ব্যাঙের সাজা", "D) পৃষ্ঠা 35"],
      "correct": "A",
      "explanation": "এবার একটা ছক আঁকিতে নিজের ভালো লাগার কথা লিডেন সমস্তিতিতে, উত্ারা খুব ভালো লাগল রাশেদ gৱ হবে, যা শুন্য করেন এর সময়ের সমান্যতাটি."
    },
    {
      "question": "রাশেদ বললেন 'নোমান' কী?\nA)
------------------------------------------------------------

👦 Student: Tell me about family.
🤖 AI Tutor (chat | english):
রাশেদ গাছটির সম্পন্নতার সমান:

রাশেদ পারবে তো:

জিজ্� Jaipur (जयपुर) - बिहारी की देशभागी महोत्सव

Jaipur,

## Step 9: Performance Summary

In [22]:
import time

# Speed test
test_q = 'পরিবার কী?'
start = time.time()
ans, _ = generate_answer(test_q, mode='chat')
elapsed = time.time() - start

print('=' * 60)
print('📊 Performance Summary')
print('=' * 60)
print(f'''
🧠 Model       : Phi-3 Mini 4k (4-bit quantized)
📚 Knowledge   : NCTB Class 3 Bengali (107 pages, 190 chunks)
🔍 Retrieval   : FAISS + Multilingual MiniLM
⚡ Speed        : {elapsed:.1f}s per response
🌐 Languages   : Bengali + English
📝 Modes       : Chat, Quiz, Explain

✅ Ready for:
   → Whisper STT integration (voice input)
   → Edge-TTS integration (voice output)
   → Django REST API wrapping
   → Flutter app connection
''')
print('=' * 60)

📊 Performance Summary

🧠 Model       : Phi-3 Mini 4k (4-bit quantized)
📚 Knowledge   : NCTB Class 3 Bengali (107 pages, 190 chunks)
🔍 Retrieval   : FAISS + Multilingual MiniLM
⚡ Speed        : 7.5s per response
🌐 Languages   : Bengali + English
📝 Modes       : Chat, Quiz, Explain

✅ Ready for:
   → Whisper STT integration (voice input)
   → Edge-TTS integration (voice output)
   → Django REST API wrapping
   → Flutter app connection

